In [44]:
import nltk

In [43]:
from nltk.corpus import wordnet 

In [3]:
syns = wordnet.synsets("program") 

In [1]:
import torch
from torch import optim
from functools import partial
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import sys
import os
import torch.nn as nn
import numpy as np



path_to_helper_files = os.path.join('..', 'py_files')
base_saved_models_dir = os.path.join('..', 'saved_models' )



sys.path.append(path_to_helper_files)


import global_variables
import dataset_helper
import nnet_models
import train_utilities

device = global_variables.device;





source_name = 'en'
target_name = 'vi'




MAX_LEN = 48
batchSize = 128


source_rnn_type  = 'lstm'



attention = True



embed_dim_array = [512]
rnn_layers_array = [2]

for embed_dim in embed_dim_array:
	for rnn_layers in rnn_layers_array:

		source_embed_dim = embed_dim
		source_hidden_size = embed_dim

		target_embed_dim= embed_dim
		target_hidden_size = 2*embed_dim
		source_rnn_layers = rnn_layers
		target_rnn_layers = rnn_layers


		if source_name == 'en' and target_name == 'vi':
				source_train_path = '../Data/iwslt-vi-en/train.tok.en'
				target_train_path = '../Data/iwslt-vi-en/train.tok.vi'

				source_val_path = '../Data/iwslt-vi-en/dev.tok.en'
				target_val_path = '../Data/iwslt-vi-en/dev.tok.vi'

				source_test_path = '../Data/iwslt-vi-en/test.tok.en'
				target_test_path = '../Data/iwslt-vi-en/test.tok.vi'

		# elif source_name == 'zh' and target_name == 'en':
		# 		target_train_path = '../Data/iwslt-zh-en/train.tok.en'
		# 		source_train_path = '../Data/iwslt-zh-en/train.tok.zh'
        #
		# 		target_val_path = '../Data/iwslt-zh-en/dev.tok.en'
		# 		source_val_path = '../Data/iwslt-zh-en/dev.tok.zh'
        #
		# 		target_test_path = '../Data/iwslt-zh-en/test.tok.en'
		# 		source_test_path = '../Data/iwslt-zh-en/test.tok.zh'
		else:
				sys.exit(source_name+'->'+target_name+' is invalid!')




		saved_models_dir = os.path.join(base_saved_models_dir, source_name+'2'+target_name)




		pth_save_folder_name = source_name+'2'+target_name+'_' + \
														'source_embed_dim='+str(source_embed_dim) +  \
														'-source_hidden_size='+str(source_hidden_size) +  \
														'-source_rnn_layers=' + str(source_rnn_layers) + \
														'-source_rnn_type='+str(source_rnn_type)+ \
														'-target_embed_dim='+str(target_embed_dim) + \
														'-target_hidden_size='+str(target_hidden_size) + \
														'-target_rnn_layers='+str(target_rnn_layers) + \
														'-attention='+str(attention);
		pth_saved_dir = os.path.join(saved_models_dir, pth_save_folder_name)



		print(pth_save_folder_name)
		sys.stdout.flush()

		saved_language_model_dir = os.path.join(saved_models_dir, 'lang_obj')



		dataset_dict = {'train': dataset_helper.LanguagePair(source_name = source_name, target_name=target_name, 
																												source_path = source_train_path, target_path = target_train_path, 
																												lang_obj_path = saved_language_model_dir), 

					 	'val': dataset_helper.LanguagePair(source_name = source_name, target_name=target_name, 
																								source_path = source_val_path, target_path = target_val_path, 
																								lang_obj_path = saved_language_model_dir, val = True), 

						'test': dataset_helper.LanguagePair(source_name = source_name, target_name=target_name, 
																								source_path = source_test_path, target_path = target_test_path, 
																								lang_obj_path = saved_language_model_dir, val = True)}																	  






		dataloader_dict = {'train': DataLoader(dataset_dict['train'], batch_size = batchSize, 
																				collate_fn = partial(dataset_helper.vocab_collate_func, MAX_LEN=MAX_LEN),
																		shuffle = True, num_workers=0), 
											'val': DataLoader(dataset_dict['val'], batch_size = 1, 
																				collate_fn = dataset_helper.vocab_collate_func_val,
																		shuffle = True, num_workers=0), 
											'test': DataLoader(dataset_dict['test'], batch_size = 1, 
																				collate_fn = dataset_helper.vocab_collate_func_val,
																		shuffle = True, num_workers=0)}





		encoder = nnet_models.EncoderRNN(dataset_dict['train'].source_lang_obj.n_words, 
																		 embed_dim = source_embed_dim, 
																		 hidden_size = source_hidden_size,
																		 rnn_layers = source_rnn_layers, 
																		 rnn_type = source_rnn_type).to(device);
																		 





		decoder = nnet_models.DecoderRNN(dataset_dict['train'].target_lang_obj.n_words, 
																								embed_dim = target_embed_dim, 
																								hidden_size = target_hidden_size, 
																								n_layers = target_rnn_layers, 
																								attention = attention).to(device)   





		encoder_optimizer = optim.SGD(encoder.parameters(), lr=0.25,nesterov=True, momentum = 0.99)
		enc_scheduler = ReduceLROnPlateau(encoder_optimizer, min_lr=1e-4,  patience=0)
		decoder_optimizer = optim.SGD(decoder.parameters(), lr=0.25,nesterov=True, momentum = 0.99)
		dec_scheduler = ReduceLROnPlateau(decoder_optimizer, min_lr=1e-4,  patience=0)





		criterion = nn.NLLLoss(ignore_index = global_variables.PAD_IDX)

en2vi_source_embed_dim=512-source_hidden_size=512-source_rnn_layers=2-source_rnn_type=lstm-target_embed_dim=512-target_hidden_size=1024-target_rnn_layers=2-attention=True


In [4]:
# Vocab Size
V = dataset_dict['train'].target_lang_obj.n_words

In [5]:
V

13294

In [21]:
dirc = 'en2vi_source_embed_dim=512-source_hidden_size=512-source_rnn_layers=2-source_rnn_type=lstm-target_embed_dim=512-target_hidden_size=1024-target_rnn_layers=2-attention=True'
encoder_name = os.path.join('..','saved_models', 'en2vi',dirc,'encoder.pth')
decoder_name = os.path.join('..','saved_models', 'en2vi',dirc,'decoder.pth')

In [22]:
encoder.load_state_dict(torch.load(encoder_name))
decoder.load_state_dict(torch.load(decoder_name))

In [24]:
len(dataloader_dict['test'].sampler)

1553

In [112]:
SOS_token = global_variables.SOS_token
EOS_token = global_variables.EOS_token
UNK_IDX = global_variables.UNK_IDX
PAD_IDX = global_variables.PAD_IDX

In [164]:

total_change = 0
loss_fun = nn.NLLLoss(ignore_index = global_variables.PAD_IDX)
true_corpus = []
pred_corpus = []
count = 0
sample = []
sample_size = 20
SAMPLE = True
for data in dataloader_dict['test']:
    count += 1
    for param in encoder.parameters():
        if param.grad is not None:
            param.grad.data.zero_()
    encoder_i = data[0].to(device)
    decoder_i = data[1].to(device)
    src_len = data[2].to(device)
    tar_len = data[3].to(device)    
    rm = 0
    out = train_utilities.encode_decode(encoder, decoder, encoder_i,decoder_i,src_len,tar_len, rand_num=rm, val = False )
    loss = loss_fun(out.float(), decoder_i.long() )
    loss.backward()
    craft_encoder_i = encoder_i.clone()
    for j in range(len(encoder_i[0])):
        cur_data = encoder_i[0][j]
        if cur_data == global_variables.PAD_IDX:
            break
        # get the grad
        grad_j = encoder.embedding.weight.grad[cur_data]
        # get synonym                
        temp = src_lang.index2word[cur_data]
        syns = wordnet.synsets(temp) 
        max_cor = 0
        flag_change = 0
        for syn in syns:
            for temp2 in syn.lemmas():
                temp3 = temp2.name()
                if temp3 in aset or temp3 == temp:
                    continue
                elif temp3 in src_lang.word2index.keys():
                    temp4 = src_lang.word2index[temp3]
                    attack_embed = encoder.embedding.weight[temp4]
                    cur_embed = encoder.embedding.weight[cur_data]
                    cur_cor = torch.sum(grad_j*(attack_embed-cur_embed))
                    if cur_cor > max_cor:
                        flag_change = 1
                        max_cor = cur_cor
                        craft_encoder_i[0][j] = temp4
        if flag_change == 1:
            total_change += 1
    if count <=sample_size:
        sample.append([train_utilities.convert_id_list_2_sent(encoder_i[0],src_lang),train_utilities.convert_id_list_2_sent(craft_encoder_i[0],src_lang) ])
    elif SAMPLE:
        break
    # Evaluate
    bs,sl = encoder_i.size()[:2]
    en_out,en_hid,en_c = encoder(craft_encoder_i,src_len)
    prev_hiddens = en_hid
    prev_cs = en_c
    decoder_input = torch.tensor([[SOS_token]]*bs).to(device)
    prev_output = torch.zeros((bs, en_out.size(-1))).to(device)
    d_out = []
    for i in range(sl*2):
        out_vocab, prev_output,prev_hiddens, prev_cs, attention_score = decoder(decoder_input,prev_output, \
                                                                                prev_hiddens,prev_cs, en_out,\
                                                                                src_len)
        topv, topi = out_vocab.topk(1)
#             decoder_input = topi.squeeze().detach().view(-1,1)
        d_out.append(topi.item())
        decoder_input = topi.squeeze().detach().view(-1,1)
        if topi.item() == EOS_token:
            break
    true_corpus.append(data[-1])
    pred_sent = train_utilities.convert_id_list_2_sent(d_out,tgt_lang)
    pred_corpus.append(pred_sent)
from bleu_score import BLEU_SCORE
bl = BLEU_SCORE()
score = bl.corpus_bleu(pred_corpus,[true_corpus],lowercase=True)[0]
print('Attack BLEU:', score)
print('Total Number of Words Changing:', total_change)

Attack BLEU: 5.471814773996189
Total Number of Words Changing: 135


In [163]:
# projected and fix the number
total_change = 0
loss_fun = nn.NLLLoss(ignore_index = global_variables.PAD_IDX)
true_corpus = []
pred_corpus = []
count = 0
sample = []
sample_size = 20
SAMPLE = False
per = 2
for data in dataloader_dict['test']:
    count += 1
    encoder_i = data[0].to(device)
    decoder_i = data[1].to(device)
    src_len = data[2].to(device)
    tar_len = data[3].to(device)    
    rm = 0
    craft_encoder_i = encoder_i.detach().clone()
    num_round = np.max((src_len//per,1))
    for _ in range(num_round):
        for param in encoder.parameters():
            if param.grad is not None:
                param.grad.data.zero_()
        max_cor = 0
        max_idx = -1
        out = train_utilities.encode_decode(encoder, decoder, craft_encoder_i,decoder_i,src_len,tar_len, rand_num=rm, val = False )
        loss = loss_fun(out.float(), decoder_i.long() )
        loss.backward()
        for j in range(len(craft_encoder_i[0])):
            cur_data = craft_encoder_i[0][j]
            if cur_data == global_variables.PAD_IDX:
                break
            # get the grad
            grad_j = encoder.embedding.weight.grad[cur_data]
            # get synonym                
            temp = src_lang.index2word[cur_data]
            syns = wordnet.synsets(temp) 
#             flag_change = 0
            for syn in syns:
                for temp2 in syn.lemmas():
                    temp3 = temp2.name()
                    if temp3 in aset or temp3 == temp:
                        continue
                    elif temp3 in src_lang.word2index.keys():
                        temp4 = src_lang.word2index[temp3]
                        attack_embed = encoder.embedding.weight[temp4]
                        cur_embed = encoder.embedding.weight[cur_data]
                        cur_cor = torch.sum(grad_j*(attack_embed-cur_embed))
                        if cur_cor > max_cor:
                            max_idx = j
                            max_cor = cur_cor
                            max_cnd = temp4
        if max_idx != -1:
            craft_encoder_i[0][max_idx] = max_cnd
            craft_encoder_i = craft_encoder_i.detach().clone()
        else:
            break
#         if flag_change == 1:
#             total_change += 1
    if count <=sample_size:
        sample.append([train_utilities.convert_id_list_2_sent(encoder_i[0],src_lang),train_utilities.convert_id_list_2_sent(craft_encoder_i[0],src_lang) ])
    elif SAMPLE:
        break
    # Evaluate
    bs,sl = encoder_i.size()[:2]
    en_out,en_hid,en_c = encoder(craft_encoder_i,src_len)
    prev_hiddens = en_hid
    prev_cs = en_c
    decoder_input = torch.tensor([[SOS_token]]*bs).to(device)
    prev_output = torch.zeros((bs, en_out.size(-1))).to(device)
    d_out = []
    for i in range(sl*2):
        out_vocab, prev_output,prev_hiddens, prev_cs, attention_score = decoder(decoder_input,prev_output, \
                                                                                prev_hiddens,prev_cs, en_out,\
                                                                                src_len)
        topv, topi = out_vocab.topk(1)
#             decoder_input = topi.squeeze().detach().view(-1,1)
        d_out.append(topi.item())
        decoder_input = topi.squeeze().detach().view(-1,1)
        if topi.item() == EOS_token:
            break
    true_corpus.append(data[-1])
    pred_sent = train_utilities.convert_id_list_2_sent(d_out,tgt_lang)
    pred_corpus.append(pred_sent)
from bleu_score import BLEU_SCORE
bl = BLEU_SCORE()
score = bl.corpus_bleu(pred_corpus,[true_corpus],lowercase=True)[0]
print('Attack BLEU:', score)
print('Total Number of Words Changing:', total_change)

Attack BLEU: 10.454769418796282
Total Number of Words Changing: 0


In [165]:
sample

[['so back in the day , when us indians wanted to travel we &apos;d jump into a UNK and we &apos;d zoom across the sky .',
  'then support inch the daylight , when us indians desire to travel we &apos;d leap into a UNK and we &apos;d zoom across the toss .'],
 ['it &apos;s just really about combining different realities .',
  'it &apos;s just very around combine unlike reality .'],
 ['at one point , your screen is filled up with leaves .',
  'at single charge , your test exist satisfy improving with provide .'],
 ['and those are the neural structures .',
  'and those equal the neural structure .'],
 ['and i was shocked and UNK to the core .',
  'and one be shock and UNK to the center .'],
 ['and yet , we have in this country this dynamic where we really don &apos;t like to talk about our problems .',
  'and even , we consume inch this nation this active where we genuinely father &apos;t wish to babble around our trouble .'],
 ['and of course , we can never have the death penalty in ger

In [143]:
count = 0
for data in dataloader_dict['test']:
    for param in encoder.parameters():
        if param.grad is not None:
            param.grad.data.zero_()
    encoder_i = data[0].to(device)
    decoder_i = data[1].to(device)
    src_len = data[2].to(device)
    tar_len = data[3].to(device)    
    rm = 0
#     out = train_utilities.encode_decode(encoder, decoder, encoder_i,decoder_i,src_len,tar_len, rand_num=rm, val = False )
#     loss = loss_fun(out.float(), decoder_i.long() )
#     loss.backward()
#     craft_encoder_i = encoder_i.clone()
    for j in range(len(encoder_i[0])):
        cur_data = encoder_i[0][j]
        if cur_data == global_variables.PAD_IDX:
            break
    count += j+1
print('Count:', count)

Count: 29536


In [146]:
# 10 percent
Attack BLEU: 16.074000340008755

2

In [ ]:
# 20 percent
Attack BLEU: 13.536007353319993

In [ ]:
# 30 percent
Attack BLEU: 11.79593366133339

In [ ]:
# 40 percent
Attack BLEU: 10.41698616667105

In [142]:
len(dataloader_dict['test'].sampler)

1553

In [ ]:
Attack BLEU: 5.6308264661308165
Total Number of Words Changing: 10967

In [161]:
sample

[['you guys knew there was a trick , didn &apos;t you .',
  'you rib knew there was a illusion , didn &apos;t you .'],
 ['what does this mean ?', 'what does this believe ?'],
 ['so in bringing back this childhood ritual , you need to go out and , in one chapter , take a picture of a flower and then tag it .',
  'so in operate back this childhood ritual , you need to go out and , in one chapter , take a see of a account and then delete it .'],
 ['i gave her my whole rap , and when i finished she looked at me and she said , &quot; mmm mmm mmm . &quot;',
  'one afford her my whole pink , and when i part she count at me and she tell , &quot; mmm mmm mmm . &quot;'],
 ['so i felt like our cellphones and our fancy watches and our cameras had stopped us from dreaming .',
  'so i felt like our cellphones and our fancy believe and our cameras had stopped us from dream .'],
 ['but , but , you know what ?', 'but , but , you know what ?'],
 ['and he was here , as you well know , in &apos; 35 .',
  

In [134]:
from bleu_score import BLEU_SCORE
bl = BLEU_SCORE()
score = bl.corpus_bleu(pred_corpus,[true_corpus],lowercase=True)[0]
print('Attack BLEU:', score)
print('Total Number of Words Changing:', total_change)

Attack BLEU: 5.809777537335008
Total Number of Words Changing: 29536


In [ ]:
score = bl.corpus_bleu(pred_corpus,[true_corpus],lowercase=True)[0]

In [116]:
import numpy as np
a = np.max(src_lang.word2index.values())

In [124]:
src_lang.n_words

16031

In [125]:
total_syns = 0
for data in dataloader_dict['test']:
    for i in range(len(data[0][0])):
        temp = src_lang.index2word[data[0][0][i]]
        syns = wordnet.synsets(temp) 
        aset = []
        for syn in syns:
            for temp2 in syn.lemmas():
                if temp2.name() in aset or temp2.name() == temp:
                    continue
                elif temp2.name() in src_lang.word2index.keys():
                    aset.append(temp2.name())
        total_syns += len(aset)

In [126]:
total_syns

106535

In [109]:
a = []

In [111]:
len(a)

0

In [130]:
import pickle
full_file_path = '../saved_models/en2vi/lang_obj/'
src_lang = pickle.load(open( full_file_path+'en_lang_obj_min_count_5.p', "rb" ))
tgt_lang = pickle.load(open( full_file_path+'vi_lang_obj_min_count_5.p', "rb" ))

In [105]:
encoder.embedding.weight[4]

tensor([-0.1967,  0.0354,  0.5990,  0.1275,  0.0795, -0.0349, -0.1683, -0.0733,
         0.2683,  0.3148, -0.2220,  0.0449,  0.3622, -0.0465,  0.4183,  0.0299,
        -0.0609,  0.0571, -0.0192, -0.2702,  0.4174,  0.0935,  0.0809,  0.0201,
        -0.1079, -0.1266, -0.2213, -0.0410,  0.1821,  0.3677, -0.0266,  0.1970,
        -0.0603, -0.0875,  0.4555,  0.2120, -0.0858, -0.0746,  0.2114,  0.0861,
        -0.0805,  0.2562, -0.3957,  0.0392,  0.2104, -0.1347, -0.0575, -0.0196,
         0.0566,  0.1692, -0.1209,  0.0712,  0.1968, -0.2708, -0.1913, -0.2343,
         0.0930,  0.0778, -0.0575, -0.0630, -0.1630, -0.1435, -0.1506,  0.4511,
         0.2082,  0.0410, -0.0060, -0.0203, -0.3231, -0.1152, -0.1980, -0.0155,
        -0.0989,  0.0861,  0.2005,  0.0113,  0.0896,  0.0330,  0.1863, -0.1990,
         0.2495,  0.0224,  0.2144,  0.2792,  0.1894, -0.0424, -0.2009,  0.2091,
         0.1105, -0.1611, -0.1534, -0.1628, -0.0760,  0.0740, -0.2061, -0.2411,
        -0.0542,  0.3449, -0.0645,  0.09

In [76]:
'Rachel' in src_lang.word2index.keys()

True

In [57]:
syns = wordnet.synsets("program") 

In [59]:
syns[1].lemmas()[0].name()

'program'

In [51]:
if syns:
    print('t')

t


In [52]:
type(sys)

module

In [65]:
syns = wordnet.synsets("good") 
aset = []
for syn in syns:
    for temp in syn.lemmas():
        if temp.name() in aset:
            break
        aset.append(temp.name())

In [66]:
aset

['good',
 'commodity',
 'trade_good',
 'full',
 'estimable',
 'beneficial',
 'adept',
 'expert',
 'dear',
 'dependable',
 'effective',
 'well',
 'thoroughly',
 'soundly']

In [147]:
a = torch.Tensor(1)

In [151]:
a.requires_grad = True

In [152]:
a.requires_grad

True

In [153]:
b = a.detach().clone()

In [154]:
b.requires_grad

False